In [3]:
from pybaseball import  playerid_lookup
from pybaseball import  statcast_pitcher
from pybaseball import  statcast


In [12]:
playerid_lookup('strider', 'spencer')['key_mlbam']

0    675911
Name: key_mlbam, dtype: int64